import libraries

In [1]:
import torch
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader
from torch import nn, optim
import os


dataset

In [2]:
def load_data(data_dir):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to fit the model
        transforms.ToTensor(),  # Convert images to PyTorch tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
    ])
    
    # Load training and testing datasets
    train_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'train'), transform=transform)
    test_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'test'), transform=transform)
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    
    return train_loader, test_loader


model

In [3]:
def create_model():
    # Load a pre-trained ResNet50 model
    model = models.resnet50(pretrained=True)
    
    # Replace the fully connected layer with a new one that outputs a single energy value
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 1),
        nn.Flatten()
    )
    
    return model


training and loss function

In [4]:
def custom_loss_function(outputs, labels):
    # Example: MSE loss that expects drones to have lower energies
    target_energies = torch.where(labels == 0, torch.zeros_like(outputs), torch.ones_like(outputs) * 1)  # drones: 0, birds: 1
    loss = torch.mean((outputs - target_energies) ** 2)
    return loss

def train_model(model, train_loader, device, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for data, labels in train_loader:
            data, labels = data.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = custom_loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Average Loss: {total_loss / len(train_loader)}')


Eval

In [5]:
def evaluate_model(model, test_loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            predicted = outputs < 0.5  # Threshold for classifying as drone
            correct += (predicted.flatten() == labels).sum().item()
            total += labels.size(0)
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy}%')


main

In [6]:
def main():
    data_dir = r'D:\Python\CS726_Project\dataset_ebm1'  # Update this path to your dataset
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    train_loader, test_loader = load_data(data_dir)
    model = create_model()
    model.to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    train_model(model, train_loader, device, optimizer, epochs=10)
    evaluate_model(model, test_loader, device)

if __name__ == "__main__":
    main()


C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Average Loss: 0.29930051786456996
Epoch 2, Average Loss: 0.2618972112140256
Epoch 3, Average Loss: 0.2538334256160759
Epoch 4, Average Loss: 0.2535139743439452
Epoch 5, Average Loss: 0.25240260032479633
Epoch 6, Average Loss: 0.2504317952308826
Epoch 7, Average Loss: 0.2545231468752473
Epoch 8, Average Loss: 0.24846959114074707
Epoch 9, Average Loss: 0.254141928014641
Epoch 10, Average Loss: 0.2577128716393145
Accuracy: 39.88326848249027%


In [9]:
def collect_scores(model, data_loader, device):
    model.eval()
    scores = []
    labels = []
    with torch.no_grad():
        for data, target in data_loader:
            data = data.to(device)
            outputs = model(data)
            scores.extend(outputs.cpu().numpy().flatten())  # Flatten and convert to list
            labels.extend(target.cpu().numpy())  # Store labels to differentiate the groups later

    return scores, labels


In [12]:
import matplotlib.pyplot as plt
def plot_histograms(scores, labels):
    drone_scores = [score for score, label in zip(scores, labels) if label == 0]
    bird_scores = [score for score, label in zip(scores, labels) if label == 1]

    plt.hist(drone_scores, bins=30, alpha=0.5, label='Drones (ID)', color='blue')
    plt.hist(bird_scores, bins=30, alpha=0.5, label='Birds (OOD)', color='red')
    plt.xlabel('Predicted Energy Scores')
    plt.ylabel('Frequency')
    plt.title('Histogram of Predicted Energy Scores')
    plt.legend()
    plt.grid(True)
    plt.show()
    plt.save()


In [ ]:
def main():
    data_dir = r'D:\Python\CS726_Project\dataset_ebm1'
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    _, test_loader = load_data(data_dir)
    model = create_model().to(device)
    
    # Assuming the model is already trained and loaded
    scores, labels = collect_scores(model, test_loader, device)
    plot_histograms(scores, labels)

if __name__ == "__main__":
    main()